In [ ]:
# Predicción de Consumo de Carbono con Gradient Boosting

Este notebook implementa un modelo de **Gradient Boosting** para predecir el consumo de carbono de una persona en función de varios factores como su tipo de cuerpo, hábitos de transporte, reciclaje, entre otros.

## Objetivo:
Utilizar un modelo de machine learning para detectar el consumo de carbono basado en los datos disponibles.



In [ ]:
#Carga de los datos que se van a usar
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from imblearn.under_sampling import RandomUnderSampler

# Leer el archivo CSV
df = pd.read_csv('carbono.csv')
df.head()

In [ ]:
Preprocesamiento de datos

In [ ]:
# Define las columnas que se desea usar 
categorical_columns = [
    'Body Type', 'Diet', 'How Often Shower', 
    'Heating Energy Source', 'Transport', 'Vehicle Type', 
    'Social Activity', 'Frequency of Traveling by Air', 
    'Waste Bag Size', 'Energy efficiency', 'Recycling', 
    'Cooking_With'
]

numeric_columns = [
    'Monthly Grocery Bill', 'Vehicle Monthly Distance Km', 
    'Waste Bag Weekly Count', 'How Long TV PC Daily Hour', 
    'How Many New Clothes Monthly', 'How Long Internet Daily Hour'
]


In [ ]:
# Configuración del preprocesamiento óptimo para el entrenamiento del modelo

onehotencoder = OneHotEncoder(sparse_output=False,drop='first') # Codificacion one-hot sin la primera categoria

scaler = MinMaxScaler(feature_range=(-1,1)) # Ecalar variables al rango [-1,1]

# Crear el transformador para aplicar las transformaciones
prepocesamiento = ColumnTransformer(
    transformers=[
        ('cat', onehotencoder, categorical_columns),  # OneHotEncoder para categóricas
        ('num', scaler, numeric_columns)  # MinMaxScaler para numéricas
    ]
)



In [ ]:
# Aplicar el preprocesamiento configurado a los datos originales
df_transformer = prepocesamiento.fit_transform(df)

# Extraer las características generadas por el OneHotEncoder
onehot_encoder_fitted = prepocesamiento.named_transformers_['cat']
cat_feature_names = onehot_encoder_fitted.get_feature_names_out(categorical_columns)

# Mantener los nombres de las columnas numéricas originales
num_feature_names = numeric_columns

# Combinar los nombres de todas las características (categóricas y numéricas)
all_feature_names = list(cat_feature_names) + num_feature_names

# Convertir el resultado del preprocesamiento en un DataFrame con nombres de columnas
df_transformer = pd.DataFrame(df_transformer, columns=all_feature_names)

# Definir la variable objetivo (CarbonEmission) separada del resto de los datos
target = df['CarbonEmission']


In [ ]:
Entrenar el modelo de Gradient Boosting

In [ ]:
# Dividir el conjunto de datos en entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(df_transformer, target, test_size=0.2, random_state=0)

# Imprimir una muestra de los datos de entrenamiento (opcional para ver cómo se ven)
# print(x_train, y_train)

# Configurar el modelo de Gradient Boosting Regressor
gmb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)

# Entrenar el modelo con los datos de entrenamiento
gmb.fit(x_train, y_train)


In [ ]:
Evaluación del modelo

In [ ]:
# Realizar predicciones en el conjunto de prueba
y_predict = gmb.predict(x_test)

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_predict)

# Calcular el error absoluto medio (MAE)
mae = mean_absolute_error(y_test, y_predict)

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_predict)

# Imprimir los resultados de las métricas de evaluación
print(f'Margen de error (MSE): {mse}')
print(f'Margen de error absoluto (MAE): {mae}')
print(f'Coeficiente de determinación (R^2): {r2}')

# Obtener las importancias de las características del modelo entrenado
features_importance = gmb.feature_importances_

# Imprimir las importancias de las características
print('Importancia de las características: ')
print(features_importance)


In [ ]:
Exportacion del modelo 

In [ ]:
# Guardar el modelo entrenado Gradient Boosting en un archivo utilizando joblib
# Esto permite reutilizar el modelo sin necesidad de volver a entrenarlo.
joblib.dump(gmb, 'boosting_model.joblib')

# Guardar el preprocesamiento (ColumnTransformer) en un archivo 
# para aplicar la misma transformación a futuros datos.
joblib.dump(prepocesamiento, 'preprocessing_pipeline.joblib')

In [ ]:
Conclucion

In [ ]:
En este proyecto, se desarrolló un modelo de Gradient Boosting Regressor para predecir las emisiones de carbono de una persona. 
El proceso incluyó un preprocesamiento completo de los datos mediante codificación de variables categóricas y normalización de las variables numéricas. 
Posteriormente, el modelo fue entrenado y evaluado usando métricas clave como el Error Cuadrático Medio (MSE), el Error Absoluto Medio (MAE) y el Coeficiente de Determinación (R²), obteniendo un R² de 0.93, lo que indica un excelente ajuste del modelo a los datos.